In [ ]:
#Install Packages
#pip install torch torchvision torchaudio
#pip install easyocr

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#import needed packages
import easyocr
from PIL import Image, ImageDraw
import cv2
import numpy as np
import re
import shutil

In [21]:
# 设置输入输出路径
input_folder = r'D:\Topo_OCR_Testing'
output_folder = r'D:\USGS_Topo\USGS_Testing_OCR'

# 确保输出路径存在
os.makedirs(output_folder, exist_ok=True)

# 初始化 EasyOCR（CPU）
reader = easyocr.Reader(['en'], gpu=False)

# 匹配纯数字
pattern = r'\d'

# 遍历所有 .tif 文件
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.tif'):
        input_path = os.path.join(input_folder, filename)
        print(f"正在处理文件: {input_path}")

        try:
        # 打开图像
            img = Image.open(input_path)
            img_gray = img.convert('L')
            img_cv = np.array(img_gray)
            _, img_thresh = cv2.threshold(img_cv, 150, 255, cv2.THRESH_BINARY)

            # OCR识别
            result = reader.readtext(np.array(img_thresh))

        # 初始化绘图对象
            draw = ImageDraw.Draw(img)
            for detection in result:
                text = detection[1]
                bbox = detection[0]
            
            # 如果是纯数字，则移除
                if re.match(pattern, text):
                    x1, y1 = int(bbox[0][0]), int(bbox[0][1])
                    x2, y2 = int(bbox[2][0]), int(bbox[2][1])
                    draw.rectangle([x1, y1, x2, y2], fill="black")

            # 保存结果
            base_name = os.path.splitext(filename)[0]
            output_filename = f"{base_name}_OCR_Remove.tif"
            output_path = os.path.join(output_folder, output_filename)
            img.save(output_path)
            print(f"已经保存至: {output_path}")
        except Exception as e:
            print(f"跳过文件 {filename}，发生错误：{e}")
            continue  # 继续下一个文件

print("全部图像处理完成。")


Using CPU. Note: This module is much faster with a GPU.


正在处理文件: D:\Topo_OCR_Testing\Academy_1946.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Academy_1946_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Brattleboro_1893.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Brattleboro_1893_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Brewer Butte_1978.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Brewer Butte_1978_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Brewton South_1959.tif
跳过文件 Brewton South_1959.tif，发生错误：y1 must be greater than or equal to y0
正在处理文件: D:\Topo_OCR_Testing\Brickeys_1984.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Brickeys_1984_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Bristol_1955.tif
跳过文件 Bristol_1955.tif，发生错误：x1 must be greater than or equal to x0
正在处理文件: D:\Topo_OCR_Testing\Britton 4 SW_1958.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Britton 4 SW_1958_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Brookston NW_1953.tif
已经保存至: D:\USGS_Topo\USGS_Testing_OCR\Brookston NW_1953_OCR_Remove.tif
正在处理文件: D:\Topo_OCR_Testing\Brownfield_1964.tif
跳过文件 Brownfield_19

In [ ]:
# 指定包含.tif文件的文件夹路径
# Path to your folder containing the .tif images
folder_path = r'D:\USGS_Topo\USGS_Testing_Tif'

output_folder = 'D:/USGS_Topo/USGS_Testing_OCR'  # 请替换为实际的输出文件夹路径

# 创建输出文件夹，如果不存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Initialize EasyOCR reader (disable GPU for CPU-only processing)
reader = easyocr.Reader(['en'], gpu=False)

# Get a list of all .tif files in the folder
tif_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.tif')]

# Process the first 20 .tif images
for i, tif_file in enumerate(tif_files[:20]):
    print(f"Processing {tif_file}...")
    # Path to your uploaded .tif image
    image_path = os.path.join(folder_path, tif_file)

    # Open the image using Pillow
    img = Image.open(image_path)

    # Convert the image to grayscale for better OCR results
    img_gray = img.convert('L')

    # Convert grayscale image to numpy array for OpenCV processing
    img_cv = np.array(img_gray)

    # Use OpenCV to process the image (thresholding to improve text extraction)
    _, img_thresh = cv2.threshold(img_cv, 150, 255, cv2.THRESH_BINARY)

    # Use EasyOCR to detect text in the image
    result = reader.readtext(np.array(img_thresh))

    # Draw over the detected numbers followed by symbols (removing them by covering with white)
    draw = ImageDraw.Draw(img)

    # Regex pattern for detecting numbers followed by symbols (degree, minute, second)
    pattern = r'\d+'
    
    # Iterate through each detected text and remove numbers followed by symbols
    for detection in result:
        text = detection[1]
        bbox = detection[0]
    
    # If the text matches our pattern (number followed by degree, minute, or second symbol)
    if re.match(pattern, text):
        # Extract the bounding box coordinates
        x1, y1 = int(bbox[0][0]), int(bbox[0][1])
        x2, y2 = int(bbox[2][0]), int(bbox[2][1])
        
        # Cover the bounding box area with white to remove the number and symbol
        draw.rectangle([x1, y1, x2, y2], fill="white")
        
 # Save the post-OCR image (with numbers removed) as a new image
    output_image_path = os.path.join(output_folder, f"OCR-Nonumber_{tif_file}")
    img.save(output_image_path)

    print(f"Processed image saved as: {output_image_path}")

Using CPU. Note: This module is much faster with a GPU.


Processing Abbeville South_1974.tif...


D:\anaconda3\envs\geoai_env\Lib\site-packages\PIL\TiffImagePlugin.py:759: UserWarning: Metadata Warning, tag 33723 had too many entries: 15, expected 1
  warnings.warn(


Processed image saved as: D:/USGS_Topo/USGS_Testing_OCR\OCR-Nonumber_Abbeville South_1974.tif
Processing Aberdeen East_1960.tif...


D:\anaconda3\envs\geoai_env\Lib\site-packages\PIL\TiffImagePlugin.py:759: UserWarning: Metadata Warning, tag 33723 had too many entries: 14, expected 1
  warnings.warn(


Processed image saved as: D:/USGS_Topo/USGS_Testing_OCR\OCR-Nonumber_Aberdeen East_1960.tif
Processing Abeytas_1952.tif...
Processed image saved as: D:/USGS_Topo/USGS_Testing_OCR\OCR-Nonumber_Abeytas_1952.tif
Processing Ada_1965.tif...


In [9]:
# 遍历文件夹中的所有.tif文件
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        # 获取输入文件路径
        input_file = os.path.join(input_folder, filename)
        
        # Open the image using Pillow
        img = Image.open(image_path)

        # Optionally, convert the image to grayscale for better OCR results
        img_gray = img.convert('L')

        # Convert grayscale image to numpy array for OpenCV processing
        img_cv = np.array(img_gray)

        # Use OpenCV to process the image (thresholding to improve text extraction)
        _, img_thresh = cv2.threshold(img_cv, 150, 255, cv2.THRESH_BINARY)

In [9]:
# Path to your uploaded .tif image
image_path = 'C:/Users/lmao/Desktop/Arcadia_1950.tif'

# Open the image using Pillow
img = Image.open(image_path)

# Convert the image to grayscale for better OCR results
img_gray = img.convert('L')

# Convert grayscale image to numpy array for OpenCV processing
img_cv = np.array(img_gray)

# Use OpenCV to process the image (thresholding to improve text extraction)
_, img_thresh = cv2.threshold(img_cv, 150, 255, cv2.THRESH_BINARY)

# Use EasyOCR to detect text in the image
result = reader.readtext(np.array(img_thresh))

# Draw over the detected numbers followed by symbols (removing them by covering with white)
draw = ImageDraw.Draw(img)

# Regex pattern for detecting numbers followed by symbols (degree, minute, second)
pattern = r'\d+'

In [11]:
# Iterate through each detected text and remove numbers followed by symbols
for detection in result:
    text = detection[1]
    bbox = detection[0]
    
    # If the text matches our pattern (number followed by degree, minute, or second symbol)
    if re.match(pattern, text):
        # Extract the bounding box coordinates
        x1, y1 = int(bbox[0][0]), int(bbox[0][1])
        x2, y2 = int(bbox[2][0]), int(bbox[2][1])
        
        # Cover the bounding box area with white to remove the number and symbol
        draw.rectangle([x1, y1, x2, y2], fill="white")
        
# Save the post-OCR image (with numbers followed by symbols removed) as a new image
output_image_path = 'C:/Users/lmao/Desktop/Arcadia_1950-OCR.tif'
img.save(output_image_path)

D:\anaconda3\envs\geoai_env\Lib\site-packages\PIL\TiffImagePlugin.py:759: UserWarning: Metadata Warning, tag 33723 had too many entries: 13, expected 1
  warnings.warn(


In [51]:
#import needed packages
import easyocr
from PIL import Image, ImageDraw
import cv2
import numpy as np
import re
import os
from datetime import datetime

# 设置输入输出路径
input_folder = r"/data/mls_data/USGS_Map_Gallery/testing_tif"
output_folder = r"/data/mls_data/USGS_Map_Gallery/testing_tif_OCR"

log_file = os.path.join(output_folder, 'ocr_log.txt')

# 确保输出目录存在
os.makedirs(output_folder, exist_ok=True)

# 初始化 EasyOCR（CPU）
reader = easyocr.Reader(['en'], gpu=True, model_storage_directory="/data/mls_data/USGS_Map_Gallery/checkpoint")

# 匹配纯数字
pattern = r'\d'

# 遍历所有 .tif 文件
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.tif'):
        input_path = os.path.join(input_folder, filename)
        print(f"正在处理文件: {input_path}")

        try:
            # 打开图像
            img = Image.open(input_path)
            img_gray = img.convert('L')
            img_cv = np.array(img_gray)
            _, img_thresh = cv2.threshold(img_cv, 150, 255, cv2.THRESH_BINARY)

            # OCR识别
            result = reader.readtext(np.array(img_thresh))

            # 初始化绘图对象
            draw = ImageDraw.Draw(img)
            for detection in result:
                text = detection[1]
                bbox = detection[0]

                # 如果是纯数字，则移除
                if re.match(pattern, text):
                    x1, y1 = int(bbox[0][0]), int(bbox[0][1])
                    x2, y2 = int(bbox[2][0]), int(bbox[2][1])
                    draw.rectangle([x1, y1, x2, y2], fill="white")

            # 保存结果
            base_name = os.path.splitext(filename)[0]
            output_filename = f"{base_name}_OCR_Remove.tif"
            output_path = os.path.join(output_folder, output_filename)
            img.save(output_path)
            print(f"已经保存至: {output_path}")
        except Exception as e:
            print(f"跳过文件 {filename}，发生错误：{e}")
            continue  # 继续下一个文件

print("全部图像处理完成。")

